In [97]:
import os
import pickle
from collections import defaultdict

In [113]:
def gtExtract(testfile, method):
    gt_confusion = defaultdict(set)
    if method == 'UDN':
        with open(testfile, 'r', encoding='utf8') as fp:
            fp.readline()
            for line in fp:
                error = line.strip().split('|||')[1].split(', ')

                for idx,cor in zip(error[::2],error[1::2]):
                    error = line[int(idx)-1]
                    gt_confusion[error].add(cor)
    return gt_confusion

In [110]:
def sysExtract(confusionset_file):
    ncm = pickle.load(open(confusionset_file, 'rb'))
    sys_confusion = defaultdict(set)
    for error, cands_info in ncm.items():
        sys_confusion[error].update(cands_info.keys())
        
    return sys_confusion

In [107]:
def versus(gt_confusion, sys_confusion):
    tp, fn, fp = 0, 0, 0
    # s=gt
    # t=sys
    for er, gtcands in gt_confusion.items():
        if er in sys_confusion:
            sys_tmpSet = set(sys_confusion[er])
            sys_tmpSet.discard(er)        
            tp += len(sys_tmpSet.intersection(gtcands))
            fn += len(gtcands.difference(sys_tmpSet))
            fp += len(sys_tmpSet.difference(gtcands))

    recall = tp/(tp+fn)
    precision = tp/(tp+fp)

    return recall, precision
#     print('Recall= {}'.format(recall))
#     print('Precision= {}'.format(precision))

In [90]:
confusionset_file = './confusionTable/0515/confu_92.pkl'
testfile = './UDN_benchmark/UDN_testdata.txt'

In [70]:
sys_confusion = pickle.load(open('co.pkl', 'rb'))

In [116]:
def converageConfusion(dataroot, testFilename, test_type='UDN'):
    filelist = {file:dataroot+file for _,_, files in os.walk(dataroot) for file in files}
    converage_file = os.path.join(dataroot, 'converage.csv')
    
    gt_c = gtExtract(testfile, test_type)
    with open(converage_file, 'w', encoding='utf8') as wp:
        wp.write('Filename,Recall,Precision\n')
        for file, path in filelist.items():
            try:
                sys_c = sysExtract(path)
            except:
                print(file)
            recall, precision = versus(gt_c, sys_c)
            wp.write('{},{},{}\n'.format(file,recall,precision))

In [112]:
testfile

'./UDN_benchmark/UDN_testdata.txt'

In [117]:
converageConfusion(dataroot, testfile, 'UDN')

converage.csv


In [95]:
dataroot = './confusionTable/0627/'

In [108]:
gt_c = gtExtract(testfile)
with open(converage_file, 'w', encoding='utf8') as wp:
    wp.write('Filename,Recall,Precision')
    for file, path in filelist.items():
        sys_c = sysExtract(path)
        recall, precision = versus(gt_c, sys_c)
        wp.write('{},{},{}'.format(file,recall,precision))

=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 
=== 


KeyboardInterrupt: 